In [ ]:
! pip freeze

### Una prima applicazione numerica dell'algebra lineare 

In questo piccolo notebook ci concentreremo su una applicazione particolarmente utile a livello numerico, degli argomenti affrontati negli scorsi notebook, ovvero, la risoluzione di un sistema di equazioni lineari tramite __matrice pseudoinversa di Moore-Penrose__. 
Abbiamo visto in precedenza come, un sistema lineare del tipo:

\begin{cases}
a_{11}x_{1} + a_{12}x_{2} + \cdots + a_{1n}x_{n} = b_{1}\\
a_{21}x_{1} + a_{22}x_{2} + \cdots + a_{2n}x_{n} = b_{2}\\
\;\vdots \\
a_{m1}x_{1} + a_{m2}x_{2} + \cdots + a_{mn}x_{n} = b_{m}
\end{cases}

Può essere rappresentato in modo compatto da tre matrici, tale per cui un sistema generico è del tipo $Ax=b$. Se assumiamo che la matrice $A$ ha tutte le colonne linearmente indipendenti, allora questo sistema può essere risolto tramite __pseudoinversa__, ovvero:$$Ax=b \Leftrightarrow A^{T}Ax = A^{T}b \Leftrightarrow x=(A^{T}A)^{-1}A^{T}b $$
in particolare, la matrice $(A^{T}A)^{-1}A^{T}$ risolve il sistema $ Ax=b $.
La matrice, utilizzando Numpy, è molto semplice da cacolare:

In [ ]:
import numpy as np 

A = np.random.randint(0, 10, (3,3)) #matrice che ha come coefficienti valori nell'intervallo [0,10)
A_T = A.T

A_pinv = np.linalg.inv(A_T @ A) @ A_T
print(A_pinv)

""" oppure utilizzando direttamente la funzione di numpy"""

A_pinv1 = np.linalg.pinv(A)
print(A_pinv1)

La pseudoinversa ha delle caratteristiche molto interessanti, infatti: sia $ A \in \mathbb{R}^{mxn} $ una matrice e sia $ A^+$ la sua pseudoinversa di __Moore–Penrose__. 
Allora, la pseudoinversa approssima l'identità nel senso che:


$ A^+ A \approx I_n \quad \text{se } m \ge n \quad $

$
A A^+ \approx I_m \quad \text{se } m \le n \quad $

In generale, la pseudoinversa soddisfa le condizioni di Moore-Penrose:
* $A A^+ A = A$ , 
* $A^+ A A^+ = A^+$, 
* $(A A^+)^T = A A^+$, 
* $(A^+ A)^T = A^+ A$.

Cerchiamo, attraverso una visualizzazione, di capire praticamente il funzionamento.

Utilizziamo la libreria più utilizzata per la visualizzazione in Python, ovvero __Matplotlib__. Basta, come già spiegato, scaricare nel venv la libreria, ovvero eseguire il comando sulla shell: ```pip install matplotlib```, una volta attivato il venv, segue un esempio della pseudoinversa di Moore-Penrose:


In [ ]:
%matplotlib inline 
"""questo comando è definito come comando magic di Ipython, 
permette di visualizzare i grafici nel notebook, piuttosto che in un'altra finestra"""

import matplotlib.pyplot as plt #importiamo la libreria matplotlib
import numpy as np 

A = np.array([[1,2,3],
              [1,2,3],
              [1,2,3]])

B = np.linalg.pinv(A)

C = A@B

print(A)
print(B)
print(A@B)

"""matshow è la funzione che ci permette di visualizzare ogni matrice come una heatmap"""

plt.matshow(A)
plt.title("Grafico della matrice originale")

plt.matshow(B)
plt.title("Grafico della matrice pesudoinversa")


plt.matshow(C)
plt.title("Grafico della matrice approssimata identità")

plt.show()

In particolare, è da notare come la matrice risultante non è esattamente quella identità, infatti la matrice iniziale $A$, non è invertibile. Al contrario:

In [ ]:
A = np.array([[1,0,0],
             [0,1,0],
              [0,0,1]])
print(A)
B = np.linalg.pinv(A)
print(B)

C = A@B
print(C)

plt.matshow(A)
plt.title("Grafico della matrice originale")

plt.matshow(B)
plt.title("Grafico della matrice pseudoinversa")

plt.matshow(C)
plt.title("Grafico della matrice approssimata identità")

plt.show()        

In questo secondo esempio la matrice contiene solo vettori linearmente indipendenti, cioè le basi canoniche del loro rispettivo spazio vettoriale, e dunque è necessariamente invertibile.

Passiamo all'ultimo argomento della nostra trattazione sulla pseudoinversa, vediamo infatti cosa accade calcolando questo particolare tipo di matrice per matrici più grandi:

In [ ]:
# non provateci nemmeno
A = np.random.randint(0,10000,(20000,20000))
B = np.linalg.pinv(A)

Questo accade perchè, il calcolo della matrice finale (anche se Numpy la calcola tramite SVD, che rende la procedura più stabile) è molto complesso, infatti questo algoritmo ha una complessità temporale dell'ordine di $O(mn^2+n^3)$.
Dobbiamo perciò utilizzare qualche stratagemma per cercare di velocizzare questo tipo di calcoli.

# Introduzione pratica a CUDA

### Accelerazione computazionale con CUDA

In questo notebook abbiamo visto come il calcolo della matrice pseudoinversa possa diventare computazionalmente costoso per matrici di grandi dimensioni. La complessità temporale dell'ordine di $O(mn² + n³)$ rende necessario trovare soluzioni per accelerare questi calcoli.

**CUDA** (Compute Unified Device Architecture) è una piattaforma di computing parallelo sviluppata da NVIDIA che permette di sfruttare la potenza delle GPU per accelerare operazioni matematiche intensive.

### Perché usare CUDA?
- **Parallelismo massivo**: Le GPU contengono migliaia di core in grado di eseguire operazioni in parallelo
- **Velocità**: Accelerazioni fino a 100x rispetto alle CPU per operazioni matriciali
- **Efficienza**: Ottimizzato specificamente per operazioni di algebra lineare



Per scaricare CUDA, basta scaricare da internet o tramite APT su linux, il toolkit per gli sviluppatori. In particolare a noi interessa una liberia per compiere al meglio delle operazioni su matrici, ovvero __Cupy__. Libreria del tutto simile a Numpy ma capace di utilizzare la computazione parallela garantita da cuda. 
Per scaricare Cupy bisogna verificare la versione di CUDA installata sul dispositivo, basta eseguire sulla shell il comando ```nvidia-smi```
e come ogni liberia, scaricare con il venv attivato tramite ```pip install cupy-cuda<versione_di_cuda_scaricata>```. Nella maggior parte dei casi basterà eseguire ```pip install cupy-cuda12x```. 

Proviamo a confrontare delle operazioni matriciali nel caso di numpy e cupy:

In [ ]:
import numpy as np 

A = np.random.randint(0,4000, (1500,1500))

B = np.linalg.pinv(A)

C = A @ B
print(A)
print(B)
print(C)


In [ ]:
import cupy as cp
"""verifichiamo se cupy è attivo con cuda"""

device = cp.is_available()
if device == True:
    print("Cuda attivo")
    print(cp.cuda.runtime.getDeviceProperties(0))

A = cp.random.randint(0,4000, (1500,1500))

B = cp.pinv(A)

C = A @ B
print(A)
print(B)
print(C)
